In [62]:
import os
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, LSTM, GRU

BATCH_SIZE = 64
EPOCHS = 100
LATENT_DIM = 256
NUM_SAMPLES = 10000

DATPATH = '../data/data.py'

import dill

In [5]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [9]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [15]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [36]:
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 'ä': 53,
 'æ': 54,
 'é': 55,
 'ë': 56}

In [17]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [27]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [32]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [46]:
sentences[0]

'preface\n\n\nsupposing that truth is a woma'

In [50]:
x[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [41]:
np.where(x[0,0]==True)

(array([42]),)

In [52]:
np.where(y[0]==True)

(array([40]),)

In [53]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [54]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [56]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 136s 678us/step - loss: 1.3591

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "y do, indeed, according to the
opinion o"
y do, indeed, according to the
opinion of the prophilous is a stronger of the proper and and conscipting the consciptined and consciptined to the strength and proper in the same as the strength of the strength is the same properious and strength and ascria. and and consciptined the sense of the fame is a proper the self-except so from the properious and endure. the passe of the sense of the strength of the strength of the prophilous and
----- diversity: 0.5
----- Generating with seed: "y do, indeed, according to the
opinion o"
y do, indeed, according to the
opinion of conscipfess to the ambition of the soul, it is so and in the subout the most standard the such fames manifess and in
when, which the operation of the stallest disod" as is every spirit, the man in the con

/home/kevin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 why by at any celaging of find ages, even the word inof severitiudes
time
use such spire
viringity at also, bor idea, in hoppers man of all eaps efficned for a philosophy, or plamentance.--to say, custurent us plato, many and and the passes "sometion!

iims of his before th
----- diversity: 1.2
----- Generating with seed: "y do, indeed, according to the
opinion o"
y do, indeed, according to the
opinion ompessus as even known after tir?
unkens well which we made him repret is dishadablait, which manifess and mind, arcept--but
mamus erogeent about therabmbbargus
dhe bad, there are remains of womans delights--for ies hepples-obthys, the
clet cutes of all god, has boened allearness, pun"es
fluenly; whore so
regree. "greeafteness we being) and bad highess his venent as human conviction aristrarous was
Epoch 2/60
200285/200285 [==============================] - 149s 746us/step - loss: 1.3530

----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "be
given if 

stands and the more of the sense"--and which is the such a lood and as the process of the moral of the subject, the strange and some sentiments of the fact that it is the sense"--the same and of the contrading to the process of the stands in the subtlest of the substable of the subjection of the sense"--and the state of the sense"--the process and the sense"--the more interests and the world, and t
----- diversity: 0.5
----- Generating with seed: ""natural." his mirroring and eternally
s"
"natural." his mirroring and eternally
sumble, fluis the strive incease and of the point, the contriction of the possession, he is what is one asteny of the more care at presents of an exception of distraining that will and the prophilomine and all the great to him which the case the spectably of an extent, in the subject, self-free of finds and will with which we say that of the works of the procession of stood with the external contri
----- diversity: 1.0
----- Generating with seed: ""natural." his 

at in allin you
the among the on the
happines of they
stupor-necessa
----- diversity: 1.2
----- Generating with seed: "nsettling books, in the latter case they"
nsettling books, in the latter case they importic; whahn they with this other-mungin crious, that love and rook a
zfaubtable opens, the overgments of the bad, religionals of unicritnos has been its philosophthohs: for labout is intendelves a man.--name that
the twest discie
long, still towards a amulate but afden henig esser taste, consequently by hanwhormals they with thingse of
this are, ot whoth everymed any tith such metaphy tornfam
Epoch 9/60
200285/200285 [==============================] - 156s 780us/step - loss: 1.3553

----- Generating text after Epoch: 8
----- diversity: 0.2
----- Generating with seed: "ions in which the basic element seems
al"
ions in which the basic element seems
all the contrary and in the sense"--and are regarded itself to the prophilly in the sense"--that is not be experience the self--it is the s

...

own heristom that is god someaday: and hand, cause rewah of mankind yet as divine european would conception of case himselfs and sound from desorgement--and age-reals
sond
and or that yiling into devil, in immere 
----- diversity: 1.2
----- Generating with seed: "_, the spheres of invariable causation,
"
_, the spheres of invariable causation,
the idion, cip bistat, and this lood; an spirits genourateall, eagly, most matressian problem, that at any desires
ritrany, was one
sublits accurations,
may therewamsehgore. we far
nothing of his contrary ba-, of clastional, in uncenalution of senses, warth
forwarle!"--catecting the every uom-"unxive more,--grut idea who know and is solen
spirite of hupos that or soce of honding renions and precle
Epoch 45/60
200285/200285 [==============================] - 129s 643us/step - loss: 1.4208

----- Generating text after Epoch: 44
----- diversity: 0.2
----- Generating with seed: "e itself injures from this motive when i"
e itself injures from this mot

 the purely conjectural nature of the soul of the state the strength of the soul and all the powerfully and the and in the strease and suppose and a strength and conscience of the supposite of the strength, as the profound that a strength, the power and as the standard and all the art and strength of the power and desire to the propless to the pastent to the sense of the sense, the powerful consequences that the sense of the senses and 
----- diversity: 0.5
----- Generating with seed: " the purely conjectural nature of the so"
 the purely conjectural nature of the soul and made got which he knows it is the cause of the saon of "the sake in the impression of the didles of the made a compulses the incarning the explares the power senses, which respects of its decisation of philosophers and command that the chary in the presut to sin the seement, by a stronger for a relations, or our passion, the angly and a sentiment of as to the moral moral
and still general a
----- diversity: 1.0
-----

who would he himself as foof of the good eamifs to the victorical
respect and a done by otherverationsterness of a sentient asvent
of taste as the utilitaition is
----- diversity: 1.2
----- Generating with seed: "lness of the type
impossible; while the "
lness of the type
impossible; while the cast of the seriousness to enound and who mamsials the phe
jewsy on order of one?
    to stall natura. a cream
which
hid, in the "soith," finally. with othercriliues the cogmends to bew      subbeity of the milago! ithers tyranny"; if lhas go reamed a sheilaugleuëæs
constraint; thorks of one; (purposertament and
enkinarion, on whatätshild: ohrands in the extanes,
but faco-rasfollte in beiner. the 
Epoch 52/60
200285/200285 [==============================] - 146s 728us/step - loss: 1.4464

----- Generating text after Epoch: 51
----- diversity: 0.2
----- Generating with seed: "because its buzzing about is
disagreeabl"
because its buzzing about is
disagreeable of the sense, the conception of the pro

estimated of a conscience. the any soul and sense--and the
promines the free morality, on a same the evolution of an among the world problem of a developed to knowledge deter of the one it is more man the consciency of witk has nature the any and self-planer and said to one of the prowes and something which its person indeed and love and the one in such such a prowed of conscience, and remain be the p
----- diversity: 1.0
----- Generating with seed: "." that is all of little value when
esti"
." that is all of little value when
estime hoph i rearm
to be
down and agreeabively plied wieve have conduct of drink" long in other, last,--whoever callical resolction, habowits also are helst is the every prejudiced, where at last to be being away bur
of
his such that,--in the our to all one deed,
no heind one eladadment their
most incritain second against instract her,
in the sign and love,
wheald vicial,
in bloudes", and very onesgc
----- diversity: 1.2
----- Generating with seed: "." that is a

in the brigts, fell the childle to totalized wrodation: thiw, but which a howld, plooted, of this downw"-justice,
insteneds. it ord, bolicp f
Epoch 59/60
200285/200285 [==============================] - 130s 649us/step - loss: 1.8331

----- Generating text after Epoch: 58
----- diversity: 0.2
----- Generating with seed: "hat the richness of inner, rational beau"
hat the richness of inner, rational beautiful belong to the same and and and and something and sense, and and preserved and distrust and and and in the sensent and actual the conscience of the self-conscience of the same and the conscience of the democratical orded the soul--and and world, the strong themselves and proper of the soul of the same and and the self-to the sensuive and profound the self-conscience, and and itself and and pr
----- diversity: 0.5
----- Generating with seed: "hat the richness of inner, rational beau"
hat the richness of inner, rational beautiby the trainful, the actual, that the masterstare the case a

In [64]:
import types
import tempfile
import keras.models

def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            keras.models.save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = keras.models.load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = keras.models.Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

make_keras_picklable()

In [66]:
with open('rnn_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [67]:
ls

__init__.py  rnn_model.pkl  rnn-testing.ipynb


In [18]:
sentences

['preface\n\n\nsupposing that truth is a woma',
 'face\n\n\nsupposing that truth is a woman--',
 'e\n\n\nsupposing that truth is a woman--wha',
 '\nsupposing that truth is a woman--what t',
 'pposing that truth is a woman--what then',
 'sing that truth is a woman--what then? i',
 'g that truth is a woman--what then? is t',
 'hat truth is a woman--what then? is ther',
 ' truth is a woman--what then? is there n',
 'uth is a woman--what then? is there not ',
 ' is a woman--what then? is there not gro',
 ' a woman--what then? is there not ground',
 'woman--what then? is there not ground\nfo',
 'an--what then? is there not ground\nfor s',
 '-what then? is there not ground\nfor susp',
 'at then? is there not ground\nfor suspect',
 'then? is there not ground\nfor suspecting',
 'n? is there not ground\nfor suspecting th',
 'is there not ground\nfor suspecting that ',
 'there not ground\nfor suspecting that all',
 're not ground\nfor suspecting that all ph',
 'not ground\nfor suspecting that al

In [19]:
next_chars

['n',
 'w',
 't',
 'h',
 '?',
 's',
 'h',
 'e',
 'o',
 'g',
 'u',
 '\n',
 'r',
 'u',
 'e',
 'i',
 ' ',
 'a',
 'a',
 ' ',
 'i',
 's',
 'h',
 's',
 'i',
 's',
 'f',
 ' ',
 ' ',
 'e',
 'h',
 'e',
 'e',
 '\n',
 'g',
 't',
 't',
 ' ',
 'v',
 'f',
 'l',
 ' ',
 ' ',
 'd',
 's',
 'n',
 'w',
 'e',
 '-',
 'a',
 't',
 ' ',
 'r',
 'b',
 '\n',
 'r',
 'u',
 'e',
 ' ',
 'd',
 'l',
 's',
 'i',
 'o',
 'u',
 't',
 'w',
 'h',
 'h',
 'h',
 'h',
 ' ',
 'v',
 'u',
 'a',
 'y',
 'a',
 '\n',
 'e',
 ' ',
 'd',
 's',
 's',
 'o',
 'r',
 'h',
 'h',
 'e',
 'e',
 ' ',
 's',
 'l',
 'd',
 'n',
 'u',
 'e',
 'l',
 'm',
 'h',
 's',
 'o',
 'w',
 'n',
 'g',
 ' ',
 'm',
 '?',
 'e',
 'a',
 'l',
 's',
 ' ',
 's',
 'e',
 'r',
 'l',
 'w',
 ' ',
 'r',
 'l',
 't',
 'b',
 'w',
 ';',
 'n',
 'a',
 'p',
 's',
 't',
 'v',
 'y',
 'i',
 ' ',
 ' ',
 'g',
 ' ',
 'a',
 's',
 'i',
 ' ',
 'd',
 'n',
 'd',
 'c',
 'r',
 'e',
 'm',
 'n',
 'i',
 '\n',
 'd',
 'd',
 'i',
 's',
 'n',
 ' ',
 ' ',
 'l',
 'f',
 ' ',
 'e',
 ' ',
 'e',
 'c',
 'f',
 's',